<a href="https://colab.research.google.com/github/safry4/MASK-RCNN/blob/main/Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import skimage.io
import skimage.draw
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndimage
import skimage.segmentation as seg
import skimage.color as color
import cv2
from skimage.segmentation import slic
from google.colab.patches import cv2_imshow

In [ ]:
def unstack(a, axis=0):
    return np.moveaxis(a, axis, 0)

In [ ]:
#convert to uint8
def convert(img, target_type_min, target_type_max, target_type):
    imin = img.min()
    imax = img.max()

    a = (target_type_max - target_type_min) / (imax - imin)
    b = target_type_max - a * imax
    new_img = (a * img + b).astype(target_type)
    return new_img

In [ ]:
def ConnectedComponentFiltered(image, threshold, minArea):
  ret, thresh = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)
  nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh, None, None, None, 8, cv2.CV_32S)
  areas = stats[1:,cv2.CC_STAT_AREA]
  FilteredImage = np.zeros((labels.shape), np.uint8)
  for i in range(0, nlabels - 1):
      if areas[i] >= minArea:
          FilteredImage[labels == i + 1] = 255
  return FilteredImage

In [ ]:
#Filter intensity
def filter_intensity(image, threshold1, threshold2, minArea):
  #outputs an intensity image with 2 thresholds for intensity
  #brightest
  ret1, thresholded1 = cv2.threshold(image, threshold1, 255, cv2.THRESH_BINARY)
  #If the pixel value is smaller than the threshold1, it is set to 0, otherwise it is set to 255. 
  nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(thresholded1, None, None, None, 8, cv2.CV_32S)
  areas = stats[1:,cv2.CC_STAT_AREA]
  FilteredImage1 = np.zeros((labels.shape), np.uint8)
  for i in range(0, nlabels - 1):
      if areas[i] >= minArea:
        FilteredImage1[labels == i + 1] = 255

  #2nd brightest
  ret2, thresholded2 = cv2.threshold(image, threshold2, 255, cv2.THRESH_BINARY)
  nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(thresholded2, None, None, None, 8, cv2.CV_32S)
  areas = stats[1:,cv2.CC_STAT_AREA]
  FilteredImage2 = np.zeros((labels.shape), np.uint8)
  for i in range(0, nlabels - 1):
      if areas[i] >= minArea:
        FilteredImage2[labels == i + 1] = 150
  
  FilteredIntensity = np.maximum(FilteredImage1, FilteredImage2)
  return FilteredIntensity